In [12]:
!pip install --upgrade datasets huggingface_hub

## Raw Sarcasm analysis data generated by GPT-4 turbo

In [14]:
import json
from datasets import Dataset
import yaml


In [15]:

CONFIG_PATH = 'data_config.yaml'
def load_config(config_path):
        with open(config_path, 'r') as file:
            return yaml.safe_load(file)
config = load_config(CONFIG_PATH)
hf_token = config['access_token']['hf_token']

In [3]:
## to read the all_seasons_data_gpt4turbo.jsonl file
all_seasons_data_gpt4turbo = []
with open("chandler_bing_sarcasm_analysis_all_seasons.jsonl", "r") as f:
    for line in f:
        all_seasons_data_gpt4turbo.append(json.loads(line))

Two episodes, `S09E13` and `S10E03` had certain issue:
* `S09E13` did not have the desired format when generated from the script using OpenAI API because the origianl data contains come unexpected tokens.
* `S10E03` missed an important scene and lines in the script.

 Manually correcting add them through intereaction with ChatGPT.

In [4]:
#S09E13
S09E13_miss = [{
    "Chandler_quote": "Was that place... The Sun?",
    "Context": "Ross enters Monica and Chandler's apartment looking excessively tanned and explains he went to the tanning place Monica suggested. Ross: I went to that tanning place your wife suggested.",
    "Sarcasm": True,
    "Reason": "Chandler uses hyperbole humorously to exaggerate the intensity of Ross's tan, implying it's as if he had been to the sun.",
    "Unlike_chandler_sarcastic": "Oh, what a subtle and natural glow you've achieved!",
    "Unlike_chandler_sincere": "That's quite a tan, Ross. Did the tanning salon have a new special?"
}]
for data in S09E13_miss:
    data.update({"episode": "S10E03"})


In [5]:
#S10E03
S10E03_miss = [{
    "Chandler_quote": "Sure! Just give me a second to get all huffy and weird like you! Do you believe that whoever did something over here last night did what they did or didn't do ...I mean come on!!",
    "Context": "Ross is trying to discuss something serious that happened last night, and Chandler mocks Ross's manner of approaching sensitive topics. Ross: Yeah! Yeah! OK! Sure! Look! Can we...can we talk about what happened here last night?",
    "Sarcasm": True,
    "Reason": "Chandler is mimicking Ross's agitated and peculiar way of speaking to highlight the absurdity of his approach, using exaggeration.",
    "Unlike_chandler_sarcastic": "Absolutely, let us delve into the deep emotional impact of last night's events with the utmost seriousness.",
    "Unlike_chandler_sincere": "Sure, Ross, I'm all ears. Let's talk about what's bothering you."
}, {
    "Chandler_quote": "No!",
    "Context": "Ross asks Chandler if he knows about an incident involving Rachel from the previous night, implying Chandler might be unaware of something important. Ross: OK you...you really don't know what I am talking about?",
    "Sarcasm": False,
    "Reason": "Chandler is genuinely uninformed about the incident Ross is referring to.",
    "Unlike_chandler_sarcastic": "Oh, I was just pretending to be oblivious to keep the conversation thrilling.",
    "Unlike_chandler_sincere": "I really have no idea, Ross, please tell me more."
}, {
    "Chandler_quote": "Our balcony? Seriously? That's so funny because I told Monica we should put lights on our balcony. And she said \"No, no. It's too cold, nobody will go out there.\" And I said \"Maybe if we put some light out there they will\"",
    "Context": "Ross tells Chandler about seeing Rachel kissing someone on their balcony. Ross: OK! Last night after the party I saw Rachel kissing that jerk from her office out on your balcony.",
    "Sarcasm": True,
    "Reason": "Chandler uses irony to comment on the coincidence of the kissing incident on the balcony, which he had previously suggested should be lit and more inviting.",
    "Unlike_chandler_sarcastic": "Oh, how utterly surprising and unforeseen an event!",
    "Unlike_chandler_sincere": "That's really surprising, Ross. Maybe we should reconsider the lighting out there."
}, {
    "Chandler_quote": "So are you gonna...talk to her?",
    "Context": "After Ross reveals seeing Rachel kissing someone on the balcony, Chandler inquires about Ross's next step regarding Rachel. Ross: even though there were NO LIGHTS!",
    "Sarcasm": False,
    "Reason": "Chandler's question is straightforward, showing concern about how Ross plans to handle the situation.",
    "Unlike_chandler_sarcastic": "Or perhaps you'll just silently stew in your feelings, as usual?",
    "Unlike_chandler_sincere": "I think it's important you talk to her and clear things up."
}, {
    "Chandler_quote": "You know when \"That's fine\" sounds true when someone yells it and spits!",
    "Context": "Ross tries to dismiss his feelings about Rachel moving on, but Chandler sarcastically comments on the believability of Ross's statement. Ross: No I'm serious. I mean she wants to date people? Fine! I don't care but...at least she could have told me. You know I...I've been putting my life on hold and just concentrating on Emma but if she wants to go out there kissing guys she barely knows, then so will I !",
    "Sarcasm": True,
    "Reason": "Chandler is mocking the way Ross is pretending to be okay with Rachel dating other people, implying that his loud and emphatic 'fine' is anything but sincere.",
    "Unlike_chandler_sarcastic": "Yes, Ross, shouting always makes things more believable.",
    "Unlike_chandler_sincere": "It sounds like you're really not okay with this, Ross. Maybe you should talk to her about how you feel."
}, {
    "Chandler_quote": "Dude, don't rub my face in your crazy single life!",
    "Context": "Ross talks about his encounters with women since becoming single, and Chandler jokingly complains about Ross flaunting his single life adventures. Ross: And then the other day on the subway a woman \"accidentally\" sat on my hand.",
    "Sarcasm": True,
    "Reason": "Chandler sarcastically pretends to be upset about Ross sharing his single life experiences, suggesting it's more pitiful than enviable.",
    "Unlike_chandler_sarcastic": "Please, continue, I live vicariously through your thrilling escapades.",
    "Unlike_chandler_sincere": "It sounds like you're having quite the interesting time being single, Ross."
}]

for data in S10E03_miss:
    data.update({"episode": "S10E03"})

In [16]:
data_complete = all_seasons_data_gpt4turbo + S10E03_miss+ S09E13_miss

with open("chandler_bing_sarcasm_analysis_all_seasons_complete.jsonl", "w") as f:
    for item in data_complete:
        f.write(json.dumps(item) + "\n")

len(data_complete)

1978

In [51]:
## make data_compelte that can be converted to a dataset with from_dict()

data_complete_dict = {
    "Chandler_quote": [item["Chandler_quote"] for item in data_complete],
    "Context": [item["Context"] for item in data_complete],
    "Sarcasm": [item["Sarcasm"] for item in data_complete],
    "Reason": [item["Reason"] for item in data_complete],
    "Unlike_chandler_sarcastic": [item["Unlike_chandler_sarcastic"] for item in data_complete],
    "Unlike_chandler_sincere": [item["Unlike_chandler_sincere"] for item in data_complete],
    "episode": [item["episode"] for item in data_complete]
}

data_complete_dataset = Dataset.from_dict(data_complete_dict)
data_complete_dataset.push_to_hub("yl2342/friends_chandler_bing_sarcasm",
                                  'default',
                                  hf_token)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.49it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/yl2342/friends_chandler_bing_sarcasm/commit/306bb5020f9e3bc4f2c0c7210b4dc65bc2962807', commit_message='Upload dataset', commit_description='', oid='306bb5020f9e3bc4f2c0c7210b4dc65bc2962807', pr_url=None, pr_revision=None, pr_num=None)

In [52]:
sarcasm = [ data['Sarcasm'] for data in data_complete_dataset]
sarcasm_count = sum(sarcasm)
total_count = len(sarcasm)
sarcasm_percentage = sarcasm_count/total_count
print(f"Total number of sarcastic comments: {sarcasm_count}")
print(f"Total number of comments: {total_count}")
print(f"Percentage of sarcastic comments: {sarcasm_percentage}")

Total number of sarcastic comments: 1167
Total number of comments: 1978
Percentage of sarcastic comments: 0.589989888776542


## Preference data

https://huggingface.co/docs/transformers/en/chat_templating

# prompt example 
`"As Chandler Bing, the witty and sarcastic character from the TV sitcom 'Friends', you have your unique sense of humor and tendency to make jokes and use sarcasm. Given the context or conversation below within the <context> sign, please respond in your unique Chandler Bing way. Append an indicator [sarcasm: True/False] at the end indicating if you used sarcasm for your response in this context. 
<context> Ross is upset about someone eating his sandwich at work and Chandler responds with a question implying the situation is overly dramatic. <context>"`


In [41]:
# act_prompt = "You are a helpful assistant who will act like Chandler Bing, the witty and sarcastic character from the TV sitcom 'Friends' who ha unique sense of humor and tendency to make jokes and use sarcasm. Given the context or conversation provided, please respond in the style of Chandler Bing. Append an indicator [sarcasm: True/False] at the end indicating if you used sarcasm for your response in this context."

In [68]:
act_prompt = "As Chandler Bing, the witty and sarcastic character from the TV sitcom 'Friends', you have your unique sense of humor and tendency to make jokes and use sarcasm. Given the context provided below, please respond in the style of Chandler Bing. "


In [69]:
# for chosen response
# for each data entry in data_complete, create a chat template where the user content is the act context + content surrounded by <context> and </context>
# and the assistant content is the Chandler_quote

# for rejected response
# for each data entry in data_complete, create a chat template where the user content is the act context + content surrounded by <context> and </context>
# and the assistant content is the Unlike_chandler_sarcastic

chosen = []
rejected = []
prompt = []
for data in data_complete:
    chosen.append(data["Chandler_quote"])
    rejected.append( data["Unlike_chandler_sarcastic"])
    prompt.append(act_prompt + " Context: " + data['Context'])

## create a dict of list that can be converted into dataset
preference_data = {
    "chosen": chosen,
    "rejected": rejected,
    "prompt": prompt
}

preference_data
preference_dataset = Dataset.from_dict(preference_data)
preference_dataset.push_to_hub(repo_id = "yl2342/friends_chandler_bing_sarcasm",
                               config_name ='preference_raw', 
                               token = hf_token)


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00, 11.79it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/yl2342/friends_chandler_bing_sarcasm/commit/8b0831d0c3b01354ebabd1cae0d270c002fc82fb', commit_message='Upload dataset', commit_description='', oid='8b0831d0c3b01354ebabd1cae0d270c002fc82fb', pr_url=None, pr_revision=None, pr_num=None)

In [70]:
# make Templates for Chat Models
# chat = [
#    {"role": "user", "content": "Hello, how are you?"},
#    {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
#    {"role": "user", "content": "I'd like to show off how chat templating works!"},
# ]

chosen = []
rejected = []
prompt = []
for data in data_complete:
    chosen.append([
        {"role": "system", "content": act_prompt},
        {"role": "user", "content": f" Context: {data['Context']}"},
        {"role": "assistant", "content": data["Chandler_quote"]}
    ])
    rejected.append([
        {"role": "system", "content": act_prompt},
        {"role": "user", "content": f" Context: {data['Context']}"},
        {"role": "assistant", "content": data["Unlike_chandler_sarcastic"]}
    ])
    prompt.append(act_prompt + f" Context: {data['Context']}")

## create a dict of list that can be converted into dataset
preference_data = {
    "chosen": chosen,
    "rejected": rejected,
    "prompt": prompt
}
preference_data

preference_dataset = Dataset.from_dict(preference_data)
preference_dataset.push_to_hub(repo_id = "yl2342/friends_chandler_bing_sarcasm",
                               config_name ='preference_chat', 
                               token = hf_token)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.17it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/yl2342/friends_chandler_bing_sarcasm/commit/b8dab3a75d5836939fc410d8aeff0df18d0ff847', commit_message='Upload dataset', commit_description='', oid='b8dab3a75d5836939fc410d8aeff0df18d0ff847', pr_url=None, pr_revision=None, pr_num=None)

In [72]:
## instruction data
# "<s>[INST] {act_prompt} {data['Context']} [/INST] {data["Chandler_quote"]} </s>"


promopt = []
completion = []
instruct = []

for data in data_complete:
    promopt.append(act_prompt + f" Context: {data['Context']}")
    completion.append(data["Chandler_quote"])
    instruct.append("<s>[INST]"+ act_prompt+ data['Context']+ "[/INST]" +data["Chandler_quote"]+  "</s>")
    

## create a dict of list that can be converted into dataset
instruct_data = {
    "promopt": promopt,
    "completion": completion,
    "instruct": instruct
}

instruct_data = Dataset.from_dict(instruct_data)
instruct_data.push_to_hub(repo_id = "yl2342/friends_chandler_bing_sarcasm",
                               config_name ='instruct', 
                               token = hf_token)


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.81it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/yl2342/friends_chandler_bing_sarcasm/commit/98af32891cacc7721d9bb5ea2c0c4fca724b6f1b', commit_message='Upload dataset', commit_description='', oid='98af32891cacc7721d9bb5ea2c0c4fca724b6f1b', pr_url=None, pr_revision=None, pr_num=None)